In [2]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 49.3 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from konlpy.tag import Kkma
# from konlpy.tag import Komoran
# from konlpy.tag import Hannanum

In [6]:
education = pd.read_csv("교육청민원.csv")
traffic = pd.read_csv("국토교통부민원.csv")
sea = pd.read_csv("해양수산부민원.csv")

In [7]:
education = education[:500]
traffic = traffic[:500]
sea = sea[:500]

In [8]:
# 질문과 내용을 합침

e_complaint = pd.DataFrame(education['qnaTitl'])
t_complaint = pd.DataFrame(traffic['qnaTitl'])
s_complaint = pd.DataFrame(sea['qnaTitl'])

In [9]:
# 라벨링

e_complaint['label'] = 0
t_complaint['label'] = 1
s_complaint['label'] = 2

In [10]:
# 합침

df = pd.concat([e_complaint, t_complaint, s_complaint], axis = 0)
df = df.reset_index().drop('index', axis = 1)

In [11]:
# 불용어 긁어오기

stopwords = []

url = 'https://www.ranks.nl/stopwords/korean'
raw = requests.get(url, verify = False)

soup = BeautifulSoup(raw.text, 'html.parser')

soup = soup.find_all('td')

for i in soup:
    for j in i:
        if j.string == None:
            continue
        else:
            stopwords.append(str(j))

stopwords.append('경우')
stopwords.append('인가요')
stopwords.append('되나요')
stopwords.append('있나요')
stopwords.append('하나요')
stopwords.append('하는')
stopwords.append('궁금합니다')
            
stopwords[:5]

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


['아', '휴', '아이구', '아이쿠', '아이고']

In [12]:
# 정규표현식으로 한글만 남기기

def clean(dataframe):
    return [re.sub(r'[^ 가-힣]', '', c) for c in dataframe]

In [13]:
df['qnaTitl'] = clean(df['qnaTitl'])

In [14]:
# 여러가지 한국어 처리 함수 만들기
# 한나눔 코모란 너무 오래걸림...

def okt(text):
    okt = Okt()
    text = okt.morphs(text)
    text = clean(text)
    return [c for c in text if len(c) > 1]

def kkma(text):
    kkma = Kkma()
    text = kkma.morphs(text)
    text = clean(text)
    return [c for c in text if len(c) > 1]

 # ㅂ니다 같은 걸 빼기 위해 clean함수를 토큰화 함수에 걍 넣음

# def komoran(text):
#     komoran = Komoran()
#     text = komoran.morphs(text)
#     text = clean(text)
#     return [c for c in text if len(c) > 1]

# def hannanum(text):
#     hannanum = Hannanum()
#     text = hannanum.morphs(text)
#     text = clean(text)
#     return [c for c in text if len(c) > 1]

In [15]:
# 불용어 사전 토큰화함....
# 

okt_stop = []

for i in stopwords:
    A = okt(i)
    for j in A:
        if len(j) > 0:
            okt_stop = okt_stop + ",".join(okt(j)).split(',')

okt_stop = list(set(okt_stop))

kkma_stop = []

for i in stopwords:
    if len(i) > 0:
        A = kkma(i)
        for j in A:
            if len(j) > 0:
                kkma_stop = kkma_stop + ",".join(kkma(j)).split(',')

kkma_stop = list(set(kkma_stop))


# hannanum_stop = []
# for i in stopwords:
#     if len(hannanum(i)) > 0:
#         hannanum_stop = hannanum_stop + ",".join(hannanum(i)).split(',')

# hannanum_stop = list(set(hannanum_stop))

# komoran_stop = []
# for i in stopwords:
#     if len(komoran(i)) > 0:
#         komoran_stop = komoran_stop + ",".join(komoran(i)).split(',')

# komoran_stop = list(set(komoran_stop))

In [16]:
X = df['qnaTitl']
y = df['label']

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1, stratify=y) 

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [21]:
tfidf = TfidfVectorizer(lowercase=False)

# param_grid = [{'vect__ngram_range': [(1, 1)],
#                'vect__stop_words': [okt_stop, None],
#                'vect__tokenizer': [okt],
#                'clf__penalty': ['l1', 'l2'],
#                'clf__C': [1.0, 10.0, 100.0]},
#               {'vect__ngram_range': [(1, 1)],
#                'vect__stop_words': [kkma_stop, None],
#                'vect__tokenizer': [kkma],
#                'clf__penalty': ['l1', 'l2'],
#                'clf__C': [1.0, 10.0, 100.0]}
#               ]

# kkma만 시도
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [kkma_stop],
               'vect__tokenizer': [kkma],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [0.1, 1.0, 10.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver = 'liblinear', random_state = 42, multi_class='auto'))])


gs_lr_tfidf = GridSearchCV(lr_tfidf,
                           param_grid,
                           scoring='accuracy',
                           cv = 5,
                           verbose = 2)

In [22]:
gs_lr_tfidf.fit(X_train, y_train)    # '비걱거리다' --(불용어 토큰화)-> '비걱거리' --(모델학습)-> '비걱'

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END clf__C=0.1, clf__penalty=l1, vect__ngram_range=(1, 1), vect__stop_words=['', '자기집', '우선', '이지만', '라면', '도달', '다음', '그리하여', '기점', '설마', '따름', '오히려', '쾅쾅', '동안', '그러나', '아울러', '일단', '어디', '한데', '까악', '의하', '너희', '운운', '으로서', '이번', '물론', '붕붕', '이리하여', '여덟', '그런', '아홉', '전부', '제각기', '실로', '에서', '만약', '영차', '동시', '기준', '요만', '이래', '마라', '저희', '이것', '마저', '그렇지만', '만일', '타인', '관련', '아서', '요컨대', '공동', '허허', '마음대로', '단지', '니까', '오르', '설사', '입장', '다섯', '이런', '입각', '다수', '무엇', '여야', '도록', '반드시', '어이', '차라리', '여부', '궁금', '때문', '항목', '지경', '인하', '당장', '쓰겁', '진짜로', '여전히', '왜냐하면', '아래', '윙윙', '헐떡헐떡', '논하', '자마자', '남짓', '비기', '하지만', '즉시', '잠깐', '그러니까', '대로', '도착', '둥둥', '쓰이', '관계', '시각', '따위', '하물며', '해보', '좍좍', '아이', '삐걱거리', '우르르', '버금', '틀림없', '언제', '그만', '비교적', '생각', '헉헉', '끙끙', '구토', '다소', '구체적', '이러하', '거의', '탕탕', '근거', '바로', '게우', '심지어', '각각', '비걱거리', '기타', '인젠', '이천', '잇따르', '좋아', '펄렁', '같이', '소생', '려고', '다시', '대하', '더욱이', '혼자

KeyboardInterrupt: ignored